### Meta

In [3]:
# Autoreload modules without having to restart the notebook kernel.
# hi bilbo
% load_ext autoreload
% autoreload 2


# Plotting code stolen from Georg's notebook.
import matplotlib.pyplot as plt

% matplotlib inline
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 32}
plt.rc('font', **font)

UsageError: Line magic function `%` not found.


### Imports

In [7]:
import sys

sys.path.insert(1, "..\\")

#import markov_clustering as mcl

# Personal libraries
import lib.cluster
import lib.files
import lib.graph

In [3]:
### Constants

In [5]:
ICP55 = '4932.YER078C'
THRESHOLD = 900
MCL_INFLATION = 2  # The parameter controlling Markov Clustering

# ICP55's inviable neighbour.
YHC1 = '4932.YLR298C'

In [ ]:
network = lib.graph.read_STRING()

In [6]:
clusters = lib.cluster.read_csv(lib.files.make_filepath_to_mcl_clusters("icp55-cc-with-inviable-proteins.csv"))
# Remove clusters of size lte 3 as is standard in the literature.
clusters = lib.cluster.remove_clusters_of_size_lte(clusters, 3)
# Make clusters into subgraphs
clusters = []